In [1]:
import numpy as np
import tensorflow as tf

# We will use the sklearn preprocessing library, as it will be easier to standardize the data.
from sklearn import preprocessing

# Load the data
raw_csv_data = np.loadtxt('1.1 Audiobooks_data.csv',delimiter=',')

# The inputs are all columns in the csv, except for the first one [:,0]
# (which is just the arbitrary customer IDs that bear no useful information),
# and the last one [:,-1] (which is our targets)

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column. That's how datasets are conventionally organized.
targets_all = raw_csv_data[:,-1]

# Balance the dataset

In [2]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

# Standardize the inputs

In [3]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

# Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

# Split the data

In [6]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. Note that each time you rerun this code, 
# you will get different values, as each time they are shuffled randomly.
# Normally you preprocess ONCE, so you need not rerun this code once it is done.
# If you rerun this whole sheet, the npzs will be overwritten with your newly preprocessed data.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1756.0 3579 0.49063984353171275
232.0 447 0.5190156599552572
249.0 448 0.5558035714285714


# Save the three datasets in *.npz

In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# Load the data

In [9]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs_npz = npz['inputs'].astype(np.float)
train_targets_npz = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs_npz = npz['inputs'].astype(np.float)
validation_targets_npz = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs_npz = npz['inputs'].astype(np.float)
test_targets_npz = npz['targets'].astype(np.int)


# Model

In [10]:
inputs_size = 10
outputs_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(outputs_size, activation='softmax'),
       
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [13]:
BATCH_SIZE = 100
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs_npz, train_targets_npz, 
          batch_size=BATCH_SIZE, epochs=max_epochs, 
          callbacks=[early_stopping],
          validation_data=(validation_inputs_npz, validation_targets_npz), 
          verbose=2, )

Epoch 1/100
36/36 - 0s - loss: 0.3074 - accuracy: 0.8265 - val_loss: 0.3435 - val_accuracy: 0.8009
Epoch 2/100
36/36 - 0s - loss: 0.3086 - accuracy: 0.8301 - val_loss: 0.3423 - val_accuracy: 0.8009
Epoch 3/100
36/36 - 0s - loss: 0.3087 - accuracy: 0.8321 - val_loss: 0.3525 - val_accuracy: 0.8076
Epoch 4/100
36/36 - 0s - loss: 0.3084 - accuracy: 0.8301 - val_loss: 0.3341 - val_accuracy: 0.8188
Epoch 5/100
36/36 - 0s - loss: 0.3095 - accuracy: 0.8270 - val_loss: 0.3300 - val_accuracy: 0.8300
Epoch 6/100
36/36 - 0s - loss: 0.3091 - accuracy: 0.8282 - val_loss: 0.3418 - val_accuracy: 0.8098
Epoch 7/100
36/36 - 0s - loss: 0.3060 - accuracy: 0.8287 - val_loss: 0.3307 - val_accuracy: 0.8188


# Test the model

In [16]:
test_loss, test_accuracy = model.evaluate(test_inputs_npz, test_targets_npz)

14/14 [==============================] - 0s 1ms/step - loss: 0.2877 - accuracy: 0.8549


In [17]:

print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.29, Test accuracy: 85.49%
